In [ ]:
import os
import pandas as pd
import cv2
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tensorflow.data import Dataset

print(cv2.__version__)


4.11.0


In [3]:
# placing the data into a pandas dataframe


def createDF(path):
    file_name = []
    animal = []
    # training = Path(path)
    print(path)
    training = os.listdir(path)
    print(training)

    for folder in training:
        folderpath = os.path.join(path,folder)
        if folder == '.gitkeep':
            continue
        files = os.listdir(folderpath)
        for file in files:
            filepath = os.path.join(folderpath,file)
            filepath = filepath.replace('\\','/')
            file_name.append(filepath)
            if(folder == 'cats'):
                animal.append('0')  
            else:
                animal.append('1')  

    print(len(file_name))
    print(len(animal))

    df = pd.DataFrame({
        'file_name': file_name,
        'animal': animal
    })
    # shuffle
    df = df.sample(frac=1).reset_index(drop=True)
    print(df.head(3))

    return df


In [12]:
train_df = createDF("../data/training_set/training_set")
train_df.to_csv("../data/csv/train_data.csv")
train_df.shape

test_df = createDF("../data/test_set/test_set")
test_df.to_csv("../data/csv/test_data.csv")



../data/training_set/training_set
['.gitkeep', 'cats', 'dogs']
8005
8005
                                           file_name animal
0  ../data/training_set/training_set/cats/cat.139...      0
1  ../data/training_set/training_set/dogs/dog.386...      1
2  ../data/training_set/training_set/cats/cat.166...      0
../data/test_set/test_set
['.gitkeep', 'cats', 'dogs']
2023
2023
                                     file_name animal
0  ../data/test_set/test_set/dogs/dog.4319.jpg      1
1  ../data/test_set/test_set/dogs/dog.4049.jpg      1
2  ../data/test_set/test_set/dogs/dog.4192.jpg      1


In [13]:
# # sirbins threading lezgoooo
# df = pd.read_csv("../data/csv/train_data.csv")

# # Take the first 50 samples and split into 6 batches
# # testing = df.head(50)

# # half of cores buot ka
# threads = math.ceil(os.cpu_count() / 2)
# arr2 = np.array_split(df, threads)
# print(f"threads {threads}")

# # Function to process a batch of images
# def process(images):
#     animal_type = []
#     image_arr = []

#     for _, row in images.iterrows():  
#         animal = row['animal']  # Get animal (0 = cat, 1 = dog)
#         animal_type.append(animal)

#         # Read and process the image
#         image = cv2.imread(row['file_name'])
#         image = cv2.resize(image, (224, 224))
#         normalized = image / 255.0  # Normalize (0-1)
#         image_arr.append(normalized)

#     return animal_type, image_arr

# # Parallel processing
# arrs = []
# with ThreadPoolExecutor(max_workers=threads) as ex:
#     futures = [ex.submit(process, batch) for batch in arr2] 

#     for future in futures:
#         arrs.append(future.result())  

# final_type = []
# final_images = []

# for animal, images in arrs:
#     final_type.extend(animal)
#     final_images.extend(images)

# # Convert to NumPy arrays
# X_test = np.array(final_images)
# y_test = np.array(final_type)

# # Save as .npy
# np.save("../data/X_test.npy", X_test)
# np.save("../data/y_test.npy", y_test)

# print(f"! Shape: {X_test.shape} {y_test.shape}")


In [ ]:
# preprocess yarn hehe

train = pd.read_csv("../data/csv/train_data.csv")

# train = pd.read_csv("../data/csv/test_data.csv")
# train = df.head(3)
# print(train)
animal_type =[]
image_arr = []
# TODO: RESEARCH THREADPOOLEXECUTOR
for i in range(len(train)):

    animal = train.iloc[i]['animal'] # gets the type of animal
    animal_type.append(animal)

    # print(train.iloc[i]['file_name'])
    image = cv2.imread(train.iloc[i]['file_name']) 

    # resize image 224 which is common and normalize to make it 0-1
    # image = cv2.resize(image,(224,224)) 
    image = cv2.resize(image,(150,150)) 
    normalized = image / 255.0
    image_arr.append(normalized)




  

# save as numpy mas efficient kaysa csv unsay reason? ambot nimo ayaw pagbuot char researching..
x = np.array(image_arr,dtype=np.float16)
y = np.array(animal_type,dtype=np.int8)

# TAPOL VERSION UNCOMMENT WHICH ONE YOU WANT

train_dataset = Dataset.from_tensor_slices((x, y)).batch(16)
Dataset.save(train_dataset,'../data/tfdata/saved_training')

# test_dataset = Dataset.from_tensor_slices((x_test, y_test)).batch(16)
# Dataset.save(test_dataset,'../data/tfdata/saved_testing')


# DEPRECATEEEED
# x_train = np.save("../data/numpy/X_train.npy",x)
# Y_train = np.save("../data/numpy/Y_train.npy",y)

# x_train = np.save("../data/numpy/X_test.npy",x)
# Y_train = np.save("../data/numpy/Y_test.npy",y)


In [ ]:

for x_batch, y_batch in train_dataset.take(1):  # Take 1 batch
    print("x_batch shape:", x_batch.shape)
    print("y_batch shape:", y_batch.shape)